In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install -q pyngrok

In [ ]:
# Step 1: นำเข้าไลบรารี
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import streamlit as st

# Step 2: โหลดชุดข้อมูล
file_path = "/content/StudentsPerformance.csv"
data = pd.read_csv(file_path)

# Step 3: การสำรวจข้อมูล
print("Dataset Head:\n", data.head())
print("\nDataset Info:")
print(data.info())
print("\nDataset Statistics:")
print(data.describe())

# Step 4: การเตรียมข้อมูล

categorical_cols = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = encoder.fit_transform(data[col])

X = data.drop(columns=['math score'])
y = data['math score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: สร้างโมเดล
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Linear activation for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Step 6: ฝึกโมเดล
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

# Step 7: Evaluate the Model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Model Evaluation - Loss: {loss}, MAE: {mae}")

# Save the trained model
model.save('math_score_predictor.h5')

# Step 8: พัฒนาแอป Streamlit
# สร้างแอป Streamlit (แอปพลิเคชันสาธิต)
def run_streamlit_app():
    st.title("Math Score Predictor")

    # Input fields for user
    gender = st.selectbox("Gender", data['gender'].unique())
    race = st.selectbox("Race/Ethnicity", data['race/ethnicity'].unique())
    parental_education = st.selectbox("Parental Level of Education", data['parental level of education'].unique())
    lunch = st.selectbox("Lunch", data['lunch'].unique())
    test_preparation = st.selectbox("Test Preparation Course", data['test preparation course'].unique())
    reading_score = st.slider("Reading Score", 0, 100, 50)
    writing_score = st.slider("Writing Score", 0, 100, 50)

    # Encode user inputs
    user_data = np.array([
        encoder.transform([gender])[0],
        encoder.transform([race])[0],
        encoder.transform([parental_education])[0],
        encoder.transform([lunch])[0],
        encoder.transform([test_preparation])[0],
        reading_score,
        writing_score
    ]).reshape(1, -1)

    # Normalize user inputs
    user_data = scaler.transform(user_data)

    # Load model and make prediction
    model = tf.keras.models.load_model('math_score_predictor.h5')
    prediction = model.predict(user_data)
    st.write(f"Predicted Math Score: {prediction[0][0]:.2f}")

# Uncomment to run Streamlit app locally
# if __name__ == "__main__":
#     run_streamlit_app()


Dataset Head:
    gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 4440.7246 - mae: 64.7548 - val_loss: 3894.5042 - val_mae: 60.4985
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3815.2742 - mae: 60.1175 - val_loss: 2668.9353 - val_mae: 49.6173
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2385.2324 - mae: 46.9712 - val_loss: 926.7648 - val_mae: 28.0289
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 653.3046 - mae: 22.9393 - val_loss: 285.0306 - val_mae: 13.1904
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 206.4853 - mae: 11.6941 - val_loss: 198.5893 - val_mae: 10.4148
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 127.6526 - mae: 9.0365 - val_loss: 156.1250 - val_mae: 9.0527
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 100.6956 - mae: 7.7874 - val_loss: 126.9189 - val_mae: 8.1878
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 78.0159 - mae: 6.9308 - val_loss: 105.8642 - val_mae: 7.5796
Epoch 9/50
50/50 ━━━━━━

Model Evaluation - Loss: 36.546417236328125, MAE: 4.637117862701416


In [ ]:
from google.colab import drive
drive.mount('/content/math_score_predictor.h5')

Mounted at /content/math_score_predictor.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/math_score_predictor.h5


MyDrive
